---
---

<center><h1> Project: Extract Transform Load.</h1></center>

---

----




In [ ]:
#ETl---Lets say we have a restaurant and it has multiple sources that is generating data everyday like dbms,json,csv,xml

# and it make predictions regarding future sales or some business decision are taken.

# now the inputs are in different formats but for a decision making process we require that the data is in single format
# like in a spreadsheet or in a database,so to bridge this gap between our raw data and clean data,we have etl process.
# etl stands for extraction,transformation and load,in simple terms it prepares raw data for decision making

# extract extracts data from various sources,this extraction can either be full or partial
# transforming data involves preprocessing of data like removing null values,duplicate values and all of this happens in a
# staging area..which is a temporary place like database where all of these transformation happens
# load is the final step where we load data into target location..this loading can be of2 types, like incremental or full
# when the data is loaded after extraction and transformation for the very first time, that is a full load
# and any subsequent loading of data,for a new entry is known as a partial load.

# the mechanism trough which etl process occur is called as etl pipeline,wecan automate etl pipeline and schedule it to run
# based on business requirement

# there are multiple rulesthat fecilitate using etl pipelines like,informatica,talend,pentaho,etc.

# undersatnding the context--letssay there is an e-commerce website on which there are 100 differrent products to purchase
# in 3 different categories- which are Ac, refrigerators,and microwave,
# the data is availablein csv files and mysql tables

#product data is stored in csv file,the data change in tis file rarely.New products are addedonce in months.
# this file has 4 different colproduct id,productname,product category,product price
# next we have user table. whenever we have new user the data is stored in this table.
# in our website every 2 secs a new user is signing up and their details have been stored
# in this tablewe have unique user id,user email,user name,source from which user came like instagram,fb,etc,isprime,signuptime
# transaction table-if any product is soldin our website,then the trx details arestored here-on an avg every5-15 secs
# a trxhappens in our website,and wehave 5 different cols--transaction id,user id ,product id,transaction time,price
# our website also has a page where people can apply for a refund
# now there are some rules regarding refund--refund will be done only if the trx is done in last 48 hrs.
# the details of refund is stored in refund table,wehave ticket id,user name,transaction id,transaction amount,ticket raise time
# the company automatically do refunds in 30mis, so we need to validate the request before initiating refund.
# so for that we will create a new table called valid refund table which has-ticket id, trx id,user id,price,ticket raise time

# so we have 1 sql file and 4 mysql tables apart fromthat we have 1 more table called accounts table which has details of
# account number of customers-- but for the scope of this project lets suppose we dont have access tothat.

# understanding requirements of etl pipeline---
# signup summary gets updated every 5 mins , and we need data added in the users table in the last 5 mins to generate the results
# this is the first pipeline that we need to build.
# another requirement is we need to generate a trx summary table-in this table we will store the no.of trxs product category wise 
# and product brand wise. It will also store the total sales brand wise and product wise.This table gets updated every 10 mins
# to calc this data we will require trx tableand product data csv file.This is the second pipeline
# another requirement is we need to update valid refund table-for that we require data from 3 different sources-
# 1 refund table, we will collect last request every 30 mins, trxs- trxids generated in last 48 hours, from valid refunds
# we will check if the data is refund or not in a last 48hrs.
# so we have 2 types of reuests valid and invalid,we will put the result in csv file and we will assign the reason as well

# invalid request csv has ticket id,user name,refund reject reason
# apart fromthat we need to update the valid trx table for which the refund req is found correct

# this is the 3rd etl pipeline, and it runs every 30 mins


# now goto sql workbench---import etl_dump.sqlscript in workbench and create allthe tables--etl_dump.sql file

# then see 1.Generate User Data.ipynb-- u donot need to understand this file fully


# keep on running 1.Generate User Data.ipynb in the mean time goto workbench

#select count(*) from users; 246 rows have been entered.

# because generate user data.ipynb is running, rows will be kept on adding in users.u

#then goto 2.generate transactions data

# in product_table.csv we have product_id,product_name,product_categoryand product_price-- we are not updating anything

# in  this file. Then we have 3. generate refund table-

# now,above we have setup a simulation environment to generate realtime data.Now,before going into etl pipeline--we
# will first undrstand our pipeline--
# for extraction we have extract_users_data() will extract data oflast 5 mins fromusers table.-- this func will pass the 
# data to transform_users_data() wich will do required transformation .Finally,load_user_summary() for loading data into
# target table.All these steps will be executed in order every 5mins.
# for the 2nd pipeline ,we need to extract product data from csv file fo which define extract_product_data().. we also
# need to extract trx data in extract_transaction_data()... data from both these functions are passed into transform_trx_data()
# and finally load_transaction_summary() will load the data into required target table--- all these steps will be executed 
# every 10 mins
# For the finalpipeline, we need to extract_transaction_data(),extract_refund_data(),extract_valid_refund_data()-- we
# will use all these result and pass it into transform_refund_data() for required trnsformation and finally load_refund_table()
# togenerate invalid request csv file and also upload vald data into refund table-- Nowthis pipeline is executed every 30 mins

# for all these pipelines we will needa single DB connection

# now datain extract_product_data() changes rarely and we donot need to extract it every 10 mins. So we will use this func
# outside the pipelin

# now we will start building etl pipeline --

# etl pipeline 1 to update user summary table--


In [ ]:
!pip install schedule

![](images/pipeline.png)


---

In [4]:
# importing the required libraries
import schedule
import time
import pandas as pd
import mysql.connector as mysql
from datetime import timedelta
import datetime

In [5]:
# connect to the database
db = mysql.connect(
    host = "localhost",
    user = "root",  ## Enter your user name here
    #password = "ABC@123", ## Enter your password here
    database = "etl", ## Enter the database name here
    #auth_plugin = "mysql_native_password",
)
# create the cursor
cursor = db.cursor()

In [ ]:
# now for the entirety of this project keep the generate user data,trx data,refund data files running until we r finished 
# with all the pipelines

In [ ]:
# defining extraction function--extract_user_data() for extraction of data
# parameters required--database connection string to connect and query database table, start time and end time to modify query
# output - panas df of extracted data

---
---

#### `EXTRACT`


---
---

In [ ]:
# defining transformation function--
#transform_user_data() it will use data from extract_user_data() oflast 5 min and do following transformation using pandas
# replace the category not available with organic in the source col,use group by func to calcno. of users in each category
# of source, use group by func to calc no. of prime users in each category of source,store all the results in a dict
# add the start and end time in the dictionary and return it.
# parameter required- df_user - user data dataframe of last 5 mins,start time and end time toupdate the o/p,so that we know
# the signup summary timeline
# it will return dictionary that will be used to import data into users summary table. 

In [6]:
# define the extract function
def extract(db):
    
    # create database cursor
    cursor = db.cursor()
    
    # current time & time 5 minutes before
    now = datetime.datetime.now()
    now_minus_5 = now - datetime.timedelta(minutes = 10)
    
    print("Extracting results between {} and {}".format(str(now_minus_5),str(now)))
    
    # command to extract the data of last 5 minutes.
    command = "SELECT * FROM users WHERE signup_time BETWEEN '{}' AND '{}'".format(now_minus_5, now)
    
    # execute the command and return the results.
    cursor.execute(command)
    data = cursor.fetchall()
    return now_minus_5, now, pd.DataFrame.from_records(data, columns= ['user_id',
                                                     'user_email',
                                                     'user_name',
                                                     'source',
                                                     'is_prime',
                                                     'signup_time'])

---

#### `Transform`

---

In [7]:
# define the tranform function
def transform(df, start_time, end_time):
    print("Transformation")
    df.source.replace("Not Available", "Organic", inplace=True)
    
    source_count = df.groupby(['source'])['user_id'].count()
    prime_count = df.groupby(['source'])['is_prime'].sum()
    
    source_count_dict = source_count.to_dict()
    for key in prime_count.to_dict():
        new_key_name = "prime_from_" + key
        source_count_dict[new_key_name] = prime_count[key]
    
    source_count_dict['start_time'] = str(start_time)
    source_count_dict['end_time'] = str(end_time)
    
    
    return source_count_dict

In [ ]:
# defining loading func- load_users_summary()-it will use the results dict from transforms_user_data and load itinto
# signup_summary table
# parameters required-result_dict-finalresults dictionary from the transform func,dbconn toupdate values in table
# it will o/p or return nothing


---

#### `LOAD`

---

In [8]:
def load(source_count_dict, db):
    print("Loading")
    cursor = db.cursor()
    command = "INSERT INTO signup_summary({col}) values{val}".format(col= ','.join(source_count_dict.keys()),
                                                                       val= tuple(source_count_dict.values()))
    cursor.execute(command)
    db.commit()

In [ ]:
# defining etl pipeline-take db conn as parameter and do foll- define timefor which u want to extract the data,
# extract latest 5 mins of users data, transform it to get users signup summary,load the data into sign_up summary table

---

#### Define the ETL Pipeline

- Step 1: Extraction
- Step 2: Transformation
- Step 3: Loading   

---

In [9]:
def job():

    db_object = create_connection()
    
    ## STEP 1: Extract
    start_time, end_time, df = extract(db_object)
    
    ## Step 2: Transform
    final_results = transform(df, start_time, end_time)
    
    ## Step 3: Load
    load(final_results, db_object)

In [10]:
schedule.every(300).seconds.do(job)

Every 300 seconds do job() (last run: [never], next run: 2023-01-18 09:31:58)

In [11]:
while True:
        schedule.run_pending() 
        time.sleep(1)

KeyboardInterrupt: 

In [ ]:
while True:
        job()
        time.sleep(300)

In [1]:
import logging

In [3]:
logging.basicConfig(filename="mylogfile.log",  level=logging.INFO)

In [6]:
for i in range(10):
    logging.info(msg= "You have a new log entry!!")